# 🫀 Heart Disease Prediction - Complete ML Project

## Project Overview
Project ini menggunakan dataset Heart Disease untuk memprediksi apakah seseorang memiliki penyakit jantung atau tidak.

**Dataset:** Heart Disease Dataset (1025 records, 14 features)

**Target:** Binary Classification (0 = No Disease, 1 = Disease)

---

## Step 1: Data Preprocessing & Cleaning

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

In [ ]:
# Load dataset
df = pd.read_csv('heart_disease.csv')

print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Dataset info
print("Dataset Information:")
print("="*50)
df.info()

In [ ]:
# Check for missing values
print("Missing Values:")
print("="*50)
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values!")

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print(f"Removed {duplicates} duplicate rows")
    print(f"New shape: {df.shape}")

In [ ]:
# Statistical summary
print("Statistical Summary:")
print("="*50)
df.describe().T

In [ ]:
# Feature descriptions
feature_descriptions = {
    'age': 'Age of patient (years)',
    'sex': 'Gender (1=male, 0=female)',
    'cp': 'Chest pain type (0-3)',
    'trestbps': 'Resting blood pressure (mm Hg)',
    'chol': 'Serum cholesterol (mg/dl)',
    'fbs': 'Fasting blood sugar > 120 mg/dl (1=true, 0=false)',
    'restecg': 'Resting ECG results (0-2)',
    'thalach': 'Maximum heart rate achieved',
    'exang': 'Exercise induced angina (1=yes, 0=no)',
    'oldpeak': 'ST depression induced by exercise',
    'slope': 'Slope of peak exercise ST segment (0-2)',
    'ca': 'Number of major vessels (0-4)',
    'thal': 'Thalassemia (0-3)',
    'target': 'Heart disease (1=yes, 0=no)'
}

print("\nFeature Descriptions:")
print("="*50)
for feat, desc in feature_descriptions.items():
    print(f"{feat:12} - {desc}")

## Step 2: Data Visualization & EDA

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
target_counts = df['target'].value_counts()
axes[0].bar(['No Disease', 'Disease'], target_counts.values, color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Target Distribution', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Count')
for i, v in enumerate(target_counts.values):
    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Pie chart
axes[1].pie(target_counts.values, labels=['No Disease', 'Disease'], 
           autopct='%1.1f%%', startangle=90, colors=['#2ecc71', '#e74c3c'])
axes[1].set_title('Target Percentage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Class Balance:")
print(f"No Disease: {target_counts[0]} ({target_counts[0]/len(df)*100:.1f}%)")
print(f"Disease: {target_counts[1]} ({target_counts[1]/len(df)*100:.1f}%)")

In [ ]:
# Age distribution by target
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
for target in [0, 1]:
    subset = df[df['target'] == target]['age']
    label = 'Disease' if target == 1 else 'No Disease'
    axes[0].hist(subset, alpha=0.6, bins=20, label=label)
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Age Distribution by Target', fontsize=14, fontweight='bold')
axes[0].legend()

# Box plot
df.boxplot(column='age', by='target', ax=axes[1])
axes[1].set_xlabel('Target (0=No Disease, 1=Disease)')
axes[1].set_ylabel('Age')
axes[1].set_title('Age vs Target', fontsize=14, fontweight='bold')
plt.suptitle('')

plt.tight_layout()
plt.show()

In [ ]:
# Categorical features analysis
categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

fig, axes = plt.subplots(2, 4, figsize=(18, 10))
axes = axes.ravel()

for idx, feature in enumerate(categorical_features):
    cross_tab = pd.crosstab(df[feature], df['target'], normalize='index') * 100
    cross_tab.plot(kind='bar', ax=axes[idx], color=['#2ecc71', '#e74c3c'])
    axes[idx].set_title(f'{feature.upper()} vs Target', fontweight='bold')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Percentage (%)')
    axes[idx].legend(['No Disease', 'Disease'])
    axes[idx].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Continuous features distribution
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, feature in enumerate(continuous_features):
    for target in [0, 1]:
        subset = df[df['target'] == target][feature]
        label = 'Disease' if target == 1 else 'No Disease'
        axes[idx].hist(subset, alpha=0.6, bins=20, label=label)
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'{feature.upper()} Distribution', fontweight='bold')
    axes[idx].legend()

# Hide last subplot
axes[-1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(14, 10))
correlation = df.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
sns.heatmap(correlation, mask=mask, annot=True, fmt='.2f', 
            cmap='coolwarm', center=0, square=True, linewidths=1)
plt.title('Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Top correlations with target
print("\nTop Correlations with Target:")
print("="*50)
target_corr = correlation['target'].drop('target').sort_values(ascending=False)
print(target_corr)

In [ ]:
# Pairplot for key features
key_features = ['age', 'trestbps', 'chol', 'thalach', 'target']
sns.pairplot(df[key_features], hue='target', palette={0: '#2ecc71', 1: '#e74c3c'})
plt.suptitle('Pairplot of Key Features', y=1.02, fontsize=16, fontweight='bold')
plt.show()

## Step 3: Modelling (Supervised Learning)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Prepare data
X = df.drop('target', axis=1)
y = df['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB()
}

# Train and evaluate models
results = []

for name, model in models.items():
    # Train
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc,
        'CV Score': cv_mean
    })

# Results dataframe
results_df = pd.DataFrame(results).sort_values('Accuracy', ascending=False)
print("\nModel Performance Comparison:")
print("="*100)
results_df

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
axes = axes.ravel()

for idx, metric in enumerate(metrics):
    results_df_sorted = results_df.sort_values(metric)
    axes[idx].barh(results_df_sorted['Model'], results_df_sorted[metric])
    axes[idx].set_xlabel(metric, fontweight='bold')
    axes[idx].set_title(f'Model Comparison - {metric}', fontsize=14, fontweight='bold')
    axes[idx].set_xlim([0, 1])
    
    # Add value labels
    for i, v in enumerate(results_df_sorted[metric]):
        axes[idx].text(v + 0.01, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Select best model based on accuracy
best_model_name = results_df.iloc[0]['Model']
best_model = models[best_model_name]

print(f"\n🏆 Best Model: {best_model_name}")
print("="*50)
print(results_df.iloc[0].to_string())

# Detailed evaluation of best model
y_pred_best = best_model.predict(X_test_scaled)
y_pred_proba_best = best_model.predict_proba(X_test_scaled)[:, 1]

print("\n\nClassification Report:")
print("="*50)
print(classification_report(y_test, y_pred_best, target_names=['No Disease', 'Disease']))

In [ ]:
# Confusion Matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix heatmap
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['No Disease', 'Disease'],
            yticklabels=['No Disease', 'Disease'])
axes[0].set_title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_best)
roc_auc = roc_auc_score(y_test, y_pred_proba_best)

axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title(f'ROC Curve - {best_model_name}', fontsize=14, fontweight='bold')
axes[1].legend(loc="lower right")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 4: Dimensionality Reduction & Feature Selection

In [ ]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier

# Feature importance using Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Feature Importance (Random Forest):")
print("="*50)
print(feature_importance)

# Visualize feature importance
plt.figure(figsize=(12, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'])
plt.xlabel('Importance Score', fontweight='bold')
plt.title('Feature Importance Analysis', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# SelectKBest feature selection
selector = SelectKBest(score_func=f_classif, k=10)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

selected_features = X.columns[selector.get_support()]
scores = selector.scores_

feature_scores = pd.DataFrame({
    'Feature': X.columns,
    'Score': scores
}).sort_values('Score', ascending=False)

print("\nFeature Selection Scores (ANOVA F-value):")
print("="*50)
print(feature_scores)
print(f"\nSelected Features (Top 10): {list(selected_features)}")

In [ ]:
# PCA Analysis
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Visualize PCA
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].bar(range(1, len(explained_variance) + 1), explained_variance)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('Scree Plot', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Cumulative variance
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('Cumulative Variance Explained', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find number of components for 95% variance
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nNumber of components for 95% variance: {n_components_95}")
print(f"Variance explained by {n_components_95} components: {cumulative_variance[n_components_95-1]:.4f}")

In [ ]:
# Compare models with different feature sets
feature_comparison = []

# Original features
best_model.fit(X_train_scaled, y_train)
y_pred = best_model.predict(X_test_scaled)
acc_original = accuracy_score(y_test, y_pred)

# Selected features (Top 10)
best_model.fit(X_train_selected, y_train)
y_pred = best_model.predict(X_test_selected)
acc_selected = accuracy_score(y_test, y_pred)

# PCA features (95% variance)
pca_optimal = PCA(n_components=n_components_95)
X_train_pca_optimal = pca_optimal.fit_transform(X_train_scaled)
X_test_pca_optimal = pca_optimal.transform(X_test_scaled)
best_model.fit(X_train_pca_optimal, y_train)
y_pred = best_model.predict(X_test_pca_optimal)
acc_pca = accuracy_score(y_test, y_pred)

comparison_df = pd.DataFrame({
    'Method': ['All Features (13)', f'Selected Features ({len(selected_features)})', f'PCA ({n_components_95} components)'],
    'Accuracy': [acc_original, acc_selected, acc_pca]
})

print("\nFeature Selection Comparison:")
print("="*50)
print(comparison_df)

# Visualize comparison
plt.figure(figsize=(10, 6))
plt.bar(comparison_df['Method'], comparison_df['Accuracy'], color=['#3498db', '#2ecc71', '#e74c3c'])
plt.ylabel('Accuracy', fontweight='bold')
plt.title('Model Performance with Different Feature Sets', fontsize=14, fontweight='bold')
plt.ylim([0.7, 1.0])
for i, v in enumerate(comparison_df['Accuracy']):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')
plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

## Step 5: Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# We'll tune the best performing model
print(f"Tuning hyperparameters for: {best_model_name}")
print("="*50)

# Define parameter grids for different models
param_grids = {
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 0.9, 1.0],
        'min_samples_split': [2, 5, 10]
    },
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    },
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
        'kernel': ['rbf', 'linear']
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    }
}

# Get the appropriate parameter grid
if best_model_name in param_grids:
    param_grid = param_grids[best_model_name]
else:
    param_grid = {}
    print(f"No predefined parameter grid for {best_model_name}")

In [ ]:
if param_grid:
    # Perform Grid Search
    print("\nPerforming Grid Search...")
    grid_search = GridSearchCV(
        estimator=best_model,
        param_grid=param_grid,
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train_scaled, y_train)
    
    print("\n\nBest Parameters:")
    print("="*50)
    for param, value in grid_search.best_params_.items():
        print(f"{param}: {value}")
    
    print(f"\nBest Cross-Validation Score: {grid_search.best_score_:.4f}")
    
    # Evaluate tuned model
    tuned_model = grid_search.best_estimator_
    y_pred_tuned = tuned_model.predict(X_test_scaled)
    
    print("\n\nTuned Model Performance:")
    print("="*50)
    print(f"Accuracy: {accuracy_score(y_test, y_pred_tuned):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred_tuned):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred_tuned):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred_tuned):.4f}")
else:
    tuned_model = best_model
    print("Using original best model")

In [ ]:
# Compare before and after tuning
if param_grid:
    comparison = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
        'Before Tuning': [
            accuracy_score(y_test, y_pred_best),
            precision_score(y_test, y_pred_best),
            recall_score(y_test, y_pred_best),
            f1_score(y_test, y_pred_best)
        ],
        'After Tuning': [
            accuracy_score(y_test, y_pred_tuned),
            precision_score(y_test, y_pred_tuned),
            recall_score(y_test, y_pred_tuned),
            f1_score(y_test, y_pred_tuned)
        ]
    })
    
    comparison['Improvement'] = comparison['After Tuning'] - comparison['Before Tuning']
    
    print("\nPerformance Comparison:")
    print("="*50)
    print(comparison)
    
    # Visualize comparison
    comparison.set_index('Metric')[['Before Tuning', 'After Tuning']].plot(
        kind='bar', figsize=(10, 6), rot=0
    )
    plt.title('Model Performance: Before vs After Tuning', fontsize=14, fontweight='bold')
    plt.ylabel('Score')
    plt.ylim([0.7, 1.0])
    plt.legend(['Before Tuning', 'After Tuning'])
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Step 6: Model Deployment Preparation

In [ ]:
import pickle

# Save the best tuned model
final_model = tuned_model if param_grid else best_model

with open('heart_disease_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Model saved: heart_disease_model.pkl")
print("Scaler saved: scaler.pkl")

In [ ]:
# Create a prediction function
def predict_heart_disease(age, sex, cp, trestbps, chol, fbs, restecg, 
                         thalach, exang, oldpeak, slope, ca, thal):
    """
    Predict heart disease based on patient features
    
    Returns:
        prediction: 0 (No Disease) or 1 (Disease)
        probability: Probability of having heart disease
    """
    # Create feature array
    features = np.array([[age, sex, cp, trestbps, chol, fbs, restecg,
                         thalach, exang, oldpeak, slope, ca, thal]])
    
    # Scale features
    features_scaled = scaler.transform(features)
    
    # Make prediction
    prediction = final_model.predict(features_scaled)[0]
    probability = final_model.predict_proba(features_scaled)[0][1]
    
    return prediction, probability

# Test the function
print("Testing prediction function:")
print("="*50)

# Example patient
test_patient = X_test.iloc[0]
pred, prob = predict_heart_disease(*test_patient.values)

print(f"Patient features: {test_patient.to_dict()}")
print(f"\nPrediction: {'Disease' if pred == 1 else 'No Disease'}")
print(f"Probability of disease: {prob:.2%}")
print(f"Actual: {'Disease' if y_test.iloc[0] == 1 else 'No Disease'}")

## Summary & Insights

In [ ]:
print("""\n
          HEART DISEASE PREDICTION - PROJECT SUMMARY           

DATASET OVERVIEW:
   • Total Records: 1025
   • Features: 13 clinical measurements
   • Target: Binary classification (Disease/No Disease)
   • Class Distribution: Balanced dataset

KEY FINDINGS:
   1. Most Important Features:
      - Number of major vessels (ca)
      - Chest pain type (cp)
      - Maximum heart rate (thalach)
      - ST depression (oldpeak)
   
   2. Strong Correlations with Target:
      - cp, thalach, slope (positive)
      - exang, oldpeak, ca (negative)

MODEL PERFORMANCE:
""")

print(f"   • Best Model: {best_model_name}")
print(f"   • Test Accuracy: {accuracy_score(y_test, y_pred_tuned if param_grid else y_pred_best):.2%}")
print(f"   • Precision: {precision_score(y_test, y_pred_tuned if param_grid else y_pred_best):.2%}")
print(f"   • Recall: {recall_score(y_test, y_pred_tuned if param_grid else y_pred_best):.2%}")
print(f"   • F1-Score: {f1_score(y_test, y_pred_tuned if param_grid else y_pred_best):.2%}")

print("""\n
DIMENSIONALITY REDUCTION:
   • Feature Selection: Top 10 features maintain high accuracy
   • PCA: {n_components_95} components explain 95% variance
   • Conclusion: All features contribute to predictions

DEPLOYMENT:
   • Model saved as: heart_disease_model.pkl
   • Scaler saved as: scaler.pkl
   • Ready for Streamlit deployment

RECOMMENDATIONS:
   1. Monitor patients with high ca, oldpeak values
   2. Chest pain type (cp) is a strong early indicator
   3. Combination of multiple factors gives best predictions
   4. Model can assist in early screening and risk assessment

      COMPLETE
""")